In [29]:
from pydantic import BaseModel

class City(BaseModel):
    name: str
    country: str
    description: str

class Itinerary(BaseModel):
    cities: list[City]

from openai import OpenAI
schema_str = Itinerary.model_json_schema()
payload = [
    {"role": "system", "content": f"You are a helpful assistant who responds ONLY in JSON as per schema :{schema_str}"},
    {"role": "user", "content": "List 5 cities with brief description"}
]

client = OpenAI(
    base_url="http://localhost:11434/v1",  # Add /v1 for OpenAI compatibility
    api_key="ollama"
)

response = client.chat.completions.create(
    model="llama3.2:latest",
    stream=False,
    messages=payload,
    # response_format={"type":"json_object"}
    # response_format=Itinerary.model_json_schema()
)
print(response.choices[0].message.content)

📋 The JSON Schema being sent to the model:
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "description": {
          "title": "Description",
          "type": "string"
        }
      },
      "required": [
        "name",
        "country",
        "description"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "Itinerary",
  "type": "object"
}

⚠️  Notice the complex structure with $defs and references!
    This can confuse the model.


Approach 1: Using schema only
{
  "cities": [
    {
      "name": "Tokyo",
      "country": "Japan",
      "description": "The capital city of Japan, known for its